In [290]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, random_split, DataLoader

# Set a random seed for reproducibility
torch.manual_seed(42)

# Load your data into a pandas DataFrame
data = pd.read_csv('./concatenated_snps.csv', index_col=0)


In [291]:
data

,chr2_192381934_C_T,chr8_129199566_G_A,"chr4_126752992_A_AAT,AATAT",chr8_29509616_A_C,chr2_121089731_T_C,chr15_75750383_T_C,chr18_24330269_A_<CN0>,chr18_24337424_C_G,chr5_131640536_A_G,chr16_4008542_CAAAAA_C,...,chr12_223587_A_<CN2>,chr12_266463_A_<CN2>,chr12_292172_T_<CN0>,chr12_293626_A_G,chr16_54682064_G_A,chr9_110303808_TAA_T,chr5_158180107_T_<CN2>,chr5_158244083_C_T,chr11_42844441_C_T,chr12_120832146_C_T
HG00096,1|1,0|1,0|1,1|1,1|0,1|0,0|0,1|1,0|1,1|1,...,0|0,0|0,0|0,0|0,1|1,0|0,0|0,0|1,0|0,0|0
HG00097,1|1,1|0,1|1,1|1,0|0,0|0,0|0,0|1,1|1,1|1,...,0|0,0|0,0|0,1|0,0|0,0|0,0|0,1|0,0|0,1|0
HG00099,1|1,0|0,0|1,0|0,0|1,0|0,0|0,1|1,0|1,1|1,...,0|0,0|0,0|0,1|0,1|1,0|0,0|0,0|1,0|0,0|0
HG00100,0|1,0|0,1|0,0|1,0|0,0|0,0|0,0|1,1|0,1|1,...,0|0,0|0,0|0,1|0,1|1,0|0,0|0,1|0,1|1,0|1
HG00101,1|1,0|0,1|0,1|1,1|0,1|1,0|0,1|1,1|1,1|1,...,0|0,0|0,0|0,0|0,0|1,0|0,0|0,1|1,0|0,0|1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1|1,0|0,0|0,1|1,0|0,0|0,0|0,0|1,0|1,0|1,...,0|0,0|0,0|0,1|0,1|1,0|1,0|0,1|1,0|0,0|0
NA21141,1|1,0|0,1|1,0|0,0|0,0|1,0|0,1|1,1|1,1|1,...,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|1,0|0
NA21142,1|0,1|0,0|0,0|1,0|0,1|0,0|0,0|1,0|1,1|1,...,0|0,0|0,0|0,0|1,0|0,0|0,0|0,1|0,0|1,1|0
NA21143,1|1,0|0,1|0,1|1,0|0,1|1,0|0,0|1,0|1,1|1,...,0|0,0|0,0|0,1|0,1|1,0|0,0|0,1|1,1|0,0|0


In [292]:
# Get the chromosome positions in PRS313 from ./positions/locations.txt
PRS313_positions = pd.read_excel('../PRS313.xlsx')

# Rename the SNPa column to have a "chr" at the beginning
PRS313_positions['SNPa'] = 'chr' + PRS313_positions['SNPa'].astype(str)



In [293]:
# Filter the data to only include the positions in PRS313 

excluded_data = data.loc[:, ~data.columns.isin(PRS313_positions['SNPa'])]


data = data.loc[:, data.columns.isin(PRS313_positions['SNPa'])]

data


,chr2_192381934_C_T,chr8_129199566_G_A,chr8_29509616_A_C,chr2_121089731_T_C,chr15_75750383_T_C,chr18_24337424_C_G,chr5_131640536_A_G,chr16_4008542_CAAAAA_C,chr16_10706580_G_A,chr9_110885479_C_T,...,chr1_51467096_CT_C,chr5_345109_T_C,chr6_152023191_G_A,chr12_96027759_A_G,chr12_293626_A_G,chr16_54682064_G_A,chr9_110303808_TAA_T,chr5_158244083_C_T,chr11_42844441_C_T,chr12_120832146_C_T
HG00096,1|1,0|1,1|1,1|0,1|0,1|1,0|1,1|1,0|0,1|1,...,1|1,0|0,0|0,0|0,0|0,1|1,0|0,0|1,0|0,0|0
HG00097,1|1,1|0,1|1,0|0,0|0,0|1,1|1,1|1,0|0,1|0,...,1|1,0|0,0|0,0|0,1|0,0|0,0|0,1|0,0|0,1|0
HG00099,1|1,0|0,0|0,0|1,0|0,1|1,0|1,1|1,0|0,1|0,...,0|0,0|0,1|0,1|0,1|0,1|1,0|0,0|1,0|0,0|0
HG00100,0|1,0|0,0|1,0|0,0|0,0|1,1|0,1|1,0|0,1|1,...,0|0,0|0,1|0,0|0,1|0,1|1,0|0,1|0,1|1,0|1
HG00101,1|1,0|0,1|1,1|0,1|1,1|1,1|1,1|1,0|0,0|1,...,0|0,0|0,1|0,0|1,0|0,0|1,0|0,1|1,0|0,0|1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1|1,0|0,1|1,0|0,0|0,0|1,0|1,0|1,0|0,1|1,...,1|0,0|0,0|1,1|1,1|0,1|1,0|1,1|1,0|0,0|0
NA21141,1|1,0|0,0|0,0|0,0|1,1|1,1|1,1|1,0|0,1|1,...,0|0,0|0,0|1,1|0,0|0,1|0,0|0,0|0,0|1,0|0
NA21142,1|0,1|0,0|1,0|0,1|0,0|1,0|1,1|1,1|0,1|1,...,1|1,0|0,0|0,0|1,0|1,0|0,0|0,1|0,0|1,1|0
NA21143,1|1,0|0,1|1,0|0,1|1,0|1,0|1,1|1,0|0,1|1,...,1|0,0|0,0|1,1|1,1|0,1|1,0|0,1|1,1|0,0|0


In [294]:
data.max().max()

'1|1'

In [295]:
excluded_data

,"chr4_126752992_A_AAT,AATAT",chr18_24330269_A_<CN0>,chr16_10699603_G_<CN0>,chr17_39054216_A_<CN2>,chr17_39237570_G_<CN0>,"chr17_39242768_T_<CN0>,<CN2>",chr2_217955891_T_<CN0>,chr22_45319942_GGGAGGGAGGGGAGGGAAGAGGGGAGGAAGGAGGAGA_G,chr6_130169321_A_<CN2>,chr15_100851435_G_<CN2>,...,"chr10_22861490_A_C,T",chr11_46198107_A_<CN2>,chr11_46202442_T_<CN2>,chr11_46249492_A_<CN2>,chr11_46317010_G_<CN2>,chr5_325948_A_<CN2>,chr12_223587_A_<CN2>,chr12_266463_A_<CN2>,chr12_292172_T_<CN0>,chr5_158180107_T_<CN2>
HG00096,0|1,0|0,0|0,0|0,0|0,0|0,0|0,1|1,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00097,1|1,0|0,0|0,0|0,0|0,0|0,1|0,1|1,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00099,0|1,0|0,0|0,0|0,0|0,0|0,0|0,1|1,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00100,1|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,...,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00101,1|0,0|0,0|0,0|0,0|0,0|0,0|0,1|1,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21141,1|1,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21142,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|1,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21143,1|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,...,2|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [296]:
# Find the positions in PRS313 that are not in the data
missing_positions = PRS313_positions.loc[~PRS313_positions['SNPa'].isin(data.columns)]

missing_positions

,SNPa,Chromosome,Positionb,Reference Allele,Effect Allele,EAFc,Overall Breast Cancerd,ER-positivee,ER-negativef,hybrid ER-positiveg,hybrid ER-negativeh
63,chr3_63887449_T_TTG,3,63887449,T,TTG,0.129725,0.0648,0.0627,0.0430,0.0648,0.0648
68,chr4_126752992_A_AAT,4,126752992,A,AAT,0.516677,-0.0377,-0.0361,-0.0638,-0.0377,-0.0377
73,chr4_187503758_A_T,4,187503758,A,T,0.447053,0.0357,0.0352,0.0194,0.0357,0.0357
75,chr4_84370124_TAA_TA,4,84370124,TAA,TA,0.532408,-0.0464,-0.0438,-0.0489,-0.0464,-0.0464
90,chr5_176134882_T_C,5,176134882,T,C,0.542245,0.0363,0.0368,0.0262,0.0363,0.0363
99,chr5_52679539_C_CA,5,52679539,C,CA,0.099758,0.0571,0.0663,0.0420,0.0571,0.0571
142,chr7_91459189_A_ATT,7,91459189,A,ATT,0.328580,0.0452,0.0439,0.0486,0.0452,0.0452
190,chr10_22861490_A_C,10,22861490,A,C,0.936979,0.0875,0.0960,0.0201,0.0960,0.0201
307,chr22_38583315_AAAAG_AAAAGAAAG,22,38583315,AAAAG,AAAAGAAAG,0.280467,-0.0471,-0.0608,0.0079,-0.0608,0.0079


In [297]:
data

,chr2_192381934_C_T,chr8_129199566_G_A,chr8_29509616_A_C,chr2_121089731_T_C,chr15_75750383_T_C,chr18_24337424_C_G,chr5_131640536_A_G,chr16_4008542_CAAAAA_C,chr16_10706580_G_A,chr9_110885479_C_T,...,chr1_51467096_CT_C,chr5_345109_T_C,chr6_152023191_G_A,chr12_96027759_A_G,chr12_293626_A_G,chr16_54682064_G_A,chr9_110303808_TAA_T,chr5_158244083_C_T,chr11_42844441_C_T,chr12_120832146_C_T
HG00096,1|1,0|1,1|1,1|0,1|0,1|1,0|1,1|1,0|0,1|1,...,1|1,0|0,0|0,0|0,0|0,1|1,0|0,0|1,0|0,0|0
HG00097,1|1,1|0,1|1,0|0,0|0,0|1,1|1,1|1,0|0,1|0,...,1|1,0|0,0|0,0|0,1|0,0|0,0|0,1|0,0|0,1|0
HG00099,1|1,0|0,0|0,0|1,0|0,1|1,0|1,1|1,0|0,1|0,...,0|0,0|0,1|0,1|0,1|0,1|1,0|0,0|1,0|0,0|0
HG00100,0|1,0|0,0|1,0|0,0|0,0|1,1|0,1|1,0|0,1|1,...,0|0,0|0,1|0,0|0,1|0,1|1,0|0,1|0,1|1,0|1
HG00101,1|1,0|0,1|1,1|0,1|1,1|1,1|1,1|1,0|0,0|1,...,0|0,0|0,1|0,0|1,0|0,0|1,0|0,1|1,0|0,0|1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1|1,0|0,1|1,0|0,0|0,0|1,0|1,0|1,0|0,1|1,...,1|0,0|0,0|1,1|1,1|0,1|1,0|1,1|1,0|0,0|0
NA21141,1|1,0|0,0|0,0|0,0|1,1|1,1|1,1|1,0|0,1|1,...,0|0,0|0,0|1,1|0,0|0,1|0,0|0,0|0,0|1,0|0
NA21142,1|0,1|0,0|1,0|0,1|0,0|1,0|1,1|1,1|0,1|1,...,1|1,0|0,0|0,0|1,0|1,0|0,0|0,1|0,0|1,1|0
NA21143,1|1,0|0,1|1,0|0,1|1,0|1,0|1,1|1,0|0,1|1,...,1|0,0|0,0|1,1|1,1|0,1|1,0|0,1|1,1|0,0|0


In [299]:
import numpy as np

def replace_non_index(value, index):
    
    alleles = value.split('|')
    
    processed_alleles = []
    for allele in alleles:
        if int(allele) == 0 or int(allele) != int(index):
            processed_alleles.append("0")
        else:
            processed_alleles.append("1")
    

    if (int(alleles[0]) > 1):
        print("PRS313 SNP: ", index)
        print("Original Data: ",alleles,"Processed Data: ", '|'.join(processed_alleles))

    return '|'.join(processed_alleles)

# Loop through the missing positions and add them to the data with the values in excluded_data
for index, row in missing_positions.iterrows():


    chromosome = row.Chromosome
    position = row.Positionb
    snp = 'chr' + str(chromosome) + '_' + str(position)
    PRS313_alternate_alleles = row["Effect Allele"]

    # Find in excluded data the columns with names that begin with snp
    snp_data = excluded_data.filter(like=snp)

    excluded_data_snp = snp_data.columns[0]

    excluded_data_snp_alternate_alleles = excluded_data_snp.split('_')[3].split(',')
    
    # Find index of PRS313_alternate_alleles in excluded_data_snp_alternate_alleles

    index = excluded_data_snp_alternate_alleles.index(PRS313_alternate_alleles) + 1

    # print("PRS313 SNP: ", PRS313_alternate_alleles)
    # print("Excluded data SNP: ", excluded_data_snp_alternate_alleles)
    # print("Alternate Allele value of PRS313 SNP in 1000 Genomes VCF: ",index)

    snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
            

    # Add snp_data to data
    data = pd.concat([data, snp_data], axis=1)

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/4177567408.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/4177567408.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/4177567408.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/4177567408.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_5

In [278]:
data

,chr2_192381934_C_T,chr8_129199566_G_A,chr8_29509616_A_C,chr2_121089731_T_C,chr15_75750383_T_C,chr18_24337424_C_G,chr5_131640536_A_G,chr16_4008542_CAAAAA_C,chr16_10706580_G_A,chr9_110885479_C_T,...,chr12:120832146:C:T_paternal,"chr3_63887449_T_TTG,TTGTG","chr4_126752992_A_AAT,AATAT","chr4_187503758_A_G,T","chr4_84370124_TAA_TA,T","chr5_176134882_T_A,C","chr5_52679539_C_CA,CAA","chr7_91459189_A_AT,ATT","chr10_22861490_A_C,T","chr22_38583315_AAAAG_AAAAGAAAG,AAAAGAAAGAAAG,A"
HG00096,1|1,0|1,1|1,1|0,1|0,1|1,0|1,1|1,0|0,1|1,...,0,0|0,0|1,0|1,0|1,0|1,0|0,1|0,1|1,0|0
HG00097,1|1,1|0,1|1,0|0,0|0,0|1,1|1,1|1,0|0,1|0,...,0,0|1,1|1,0|1,1|0,1|0,0|0,1|0,1|1,0|0
HG00099,1|1,0|0,0|0,0|1,0|0,1|1,0|1,1|1,0|0,1|0,...,0,0|0,0|1,1|1,0|0,0|0,0|0,1|1,1|1,1|1
HG00100,0|1,0|0,0|1,0|0,0|0,0|1,1|0,1|1,0|0,1|1,...,1,1|0,1|0,1|0,1|1,1|1,0|0,0|1,0|1,0|0
HG00101,1|1,0|0,1|1,1|0,1|1,1|1,1|1,1|1,0|0,0|1,...,1,0|1,1|0,1|0,1|0,1|0,0|0,0|0,1|1,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1|1,0|0,1|1,0|0,0|0,0|1,0|1,0|1,0|0,1|1,...,0,0|0,0|0,0|1,1|0,1|0,0|0,0|0,1|1,1|0
NA21141,1|1,0|0,0|0,0|0,0|1,1|1,1|1,1|1,0|0,1|1,...,0,0|0,1|1,0|0,1|0,1|1,0|0,1|1,1|1,0|1
NA21142,1|0,1|0,0|1,0|0,1|0,0|1,0|1,1|1,1|0,1|1,...,0,0|1,0|0,0|1,0|1,1|1,0|0,0|1,1|1,0|1
NA21143,1|1,0|0,1|1,0|0,1|1,0|1,0|1,1|1,0|0,1|1,...,0,0|0,1|0,0|1,1|1,0|0,1|0,1|0,0|1,1|0


In [288]:
# Split into maternal/paternal columns
new_cols = []
for col in data.columns:
    maternal_col = f"{col.replace('_', ':')}_maternal"
    paternal_col = f"{col.replace('_', ':')}_paternal"
    new_cols.extend([maternal_col, paternal_col])

# Update the new columns with the desired values
for col in data.columns:
    maternal_col = f"{col.replace('_', ':')}_maternal"
    paternal_col = f"{col.replace('_', ':')}_paternal"
    print(data[col])
    data[maternal_col] = [int(allele.split("|")[0]) for allele in data[col]]
    data[paternal_col] = [int(allele.split("|")[1]) for allele in data[col]]

# Reorder the columns
data = data[new_cols]

HG00096    1|1
HG00097    1|1
HG00099    1|1
HG00100    0|1
HG00101    1|1
          ... 
NA21137    1|1
NA21141    1|1
NA21142    1|0
NA21143    1|1
NA21144    1|1
Name: chr2_192381934_C_T, Length: 2504, dtype: object
HG00096    0|1
HG00097    1|0
HG00099    0|0
HG00100    0|0
HG00101    0|0
          ... 
NA21137    0|0
NA21141    0|0
NA21142    1|0
NA21143    0|0
NA21144    0|0
Name: chr8_129199566_G_A, Length: 2504, dtype: object
HG00096    1|1
HG00097    1|1
HG00099    0|0
HG00100    0|1
HG00101    1|1
          ... 
NA21137    1|1
NA21141    0|0
NA21142    0|1
NA21143    1|1
NA21144    0|1
Name: chr8_29509616_A_C, Length: 2504, dtype: object
HG00096    1|0
HG00097    0|0
HG00099    0|1
HG00100    0|0
HG00101    1|0
          ... 
NA21137    0|0
NA21141    0|0
NA21142    0|0
NA21143    0|0
NA21144    0|0
Name: chr2_121089731_T_C, Length: 2504, dtype: object
HG00096    1|0
HG00097    0|0
HG00099    0|0
HG00100    0|0
HG00101    1|1
          ... 
NA21137    0|0
NA21141    0|1
NA211

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[maternal_col] = [int(allele.split("|")[0]) for allele in data[col]]
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[paternal_col] = [int(allele.split("|")[1]) for allele in data[col]]
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:13: SettingWithCopyWarning: 
A v

HG00096    0|1
HG00097    0|0
HG00099    1|1
HG00100    0|1
HG00101    0|0
          ... 
NA21137    1|1
NA21141    0|0
NA21142    0|0
NA21143    1|0
NA21144    1|0
Name: chr2_39699510_C_CT, Length: 2504, dtype: object
HG00096    0|0
HG00097    0|0
HG00099    0|0
HG00100    0|0
HG00101    0|0
          ... 
NA21137    0|0
NA21141    0|0
NA21142    0|0
NA21143    0|0
NA21144    0|0
Name: chr1_242034263_A_G, Length: 2504, dtype: object
HG00096    0|0
HG00097    1|0
HG00099    0|1
HG00100    0|0
HG00101    0|0
          ... 
NA21137    1|0
NA21141    0|0
NA21142    0|0
NA21143    0|0
NA21144    0|0
Name: chr17_53209774_A_C, Length: 2504, dtype: object
HG00096    1|0
HG00097    0|0
HG00099    0|0
HG00100    1|0
HG00101    0|0
          ... 
NA21137    0|0
NA21141    0|0
NA21142    0|0
NA21143    0|0
NA21144    1|0
Name: chr2_10138983_T_C, Length: 2504, dtype: object
HG00096    0|0
HG00097    0|0
HG00099    0|0
HG00100    0|1
HG00101    0|0
          ... 
NA21137    0|0
NA21141    0|0
NA211

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[maternal_col] = [int(allele.split("|")[0]) for allele in data[col]]
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[paternal_col] = [int(allele.split("|")[1]) for allele in data[col]]
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_56198/164808398.py:13: PerformanceWarning: DataFrame is highly fragmented

In [289]:
data

,chr2:192381934:C:T_maternal,chr2:192381934:C:T_paternal,chr8:129199566:G:A_maternal,chr8:129199566:G:A_paternal,chr8:29509616:A:C_maternal,chr8:29509616:A:C_paternal,chr2:121089731:T:C_maternal,chr2:121089731:T:C_paternal,chr15:75750383:T:C_maternal,chr15:75750383:T:C_paternal,...,chr16:54682064:G:A_maternal,chr16:54682064:G:A_paternal,chr9:110303808:TAA:T_maternal,chr9:110303808:TAA:T_paternal,chr5:158244083:C:T_maternal,chr5:158244083:C:T_paternal,chr11:42844441:C:T_maternal,chr11:42844441:C:T_paternal,chr12:120832146:C:T_maternal,chr12:120832146:C:T_paternal
HG00096,1,1,0,1,1,1,1,0,1,0,...,1,1,0,0,0,1,0,0,0,0
HG00097,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
HG00099,1,1,0,0,0,0,0,1,0,0,...,1,1,0,0,0,1,0,0,0,0
HG00100,0,1,0,0,0,1,0,0,0,0,...,1,1,0,0,1,0,1,1,0,1
HG00101,1,1,0,0,1,1,1,0,1,1,...,0,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,1,0,0,0,0
NA21141,1,1,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
NA21142,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
NA21143,1,1,0,0,1,1,0,0,1,1,...,1,1,0,0,1,1,1,0,0,0


In [108]:
# Get the number of rows where a value of greater than 1 appeared
data.loc[(data > 1).any(axis=1)]


,chr2:192381934_maternal,chr2:192381934_paternal,chr8:129199566_maternal,chr8:129199566_paternal,chr4:126752992_maternal,chr4:126752992_paternal,chr8:29509616_maternal,chr8:29509616_paternal,chr2:121089731_maternal,chr2:121089731_paternal,...,chr16:54682064_maternal,chr16:54682064_paternal,chr9:110303808_maternal,chr9:110303808_paternal,chr5:158244083_maternal,chr5:158244083_paternal,chr11:42844441_maternal,chr11:42844441_paternal,chr12:120832146_maternal,chr12:120832146_paternal
HG00096,1,1,0,1,0,1,1,1,1,0,...,1,1,0,0,0,1,0,0,0,0
HG00097,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,1,0,0,0,1,0
HG00099,1,1,0,0,0,1,0,0,0,1,...,1,1,0,0,0,1,0,0,0,0
HG00100,0,1,0,0,1,0,0,1,0,0,...,1,1,0,0,1,0,1,1,0,1
HG00101,1,1,0,0,1,0,1,1,1,0,...,0,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,1,1,0,0,0,0,1,1,0,0,...,1,1,0,1,1,1,0,0,0,0
NA21141,1,1,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
NA21142,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,1,0
NA21143,1,1,0,0,1,0,1,1,0,0,...,1,1,0,0,1,1,1,0,0,0


In [109]:
# Get the number of columns where a value of greater than 1 appeared

data.loc[:, (data > 1).any()]

,chr4:126752992_maternal,chr4:126752992_paternal,chr22:38583315_maternal,chr22:38583315_paternal,chr3:63887449_maternal,chr3:63887449_paternal,chr4:84370124_maternal,chr4:84370124_paternal,chr5:176134882_maternal,chr5:176134882_paternal,chr4:187503758_maternal,chr4:187503758_paternal,chr5:52679539_maternal,chr5:52679539_paternal,chr7:91459189_maternal,chr7:91459189_paternal,chr10:22861490_maternal,chr10:22861490_paternal
HG00096,0,1,0,0,0,0,0,1,0,2,0,2,0,2,2,0,1,1
HG00097,1,1,0,0,0,1,1,0,2,0,1,2,0,2,2,0,1,1
HG00099,0,1,1,1,0,0,0,0,0,0,2,2,0,0,2,2,1,1
HG00100,1,0,0,0,1,0,1,1,2,2,2,0,0,0,1,2,0,1
HG00101,1,0,0,0,0,1,1,0,2,0,2,0,2,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,0,0,1,0,0,0,1,0,2,0,0,2,0,0,1,0,1,1
NA21141,1,1,0,1,0,0,1,0,2,2,0,0,0,0,2,2,1,1
NA21142,0,0,0,1,0,1,0,1,2,2,0,2,0,0,0,2,1,1
NA21143,1,0,1,0,0,0,1,1,0,0,0,2,1,0,2,0,2,1


In [41]:
# # Replace all the values greater than 1 with 0
# data[data > 1] = 0
# # Find positions where the value is greater than 2
# positions_greater_than_2 = np.where(data.values > 1)

# # Get column names for columns with values > 2
# cols_greater_than_2 = [data.columns[col] for col in np.unique(positions_greater_than_2[1])]

# # Display only columns with values > 2
# print("Columns where values exist greater than 2:")
# print(data[cols_greater_than_2])

Columns where values exist greater than 2:
Empty DataFrame
Columns: []
Index: [HG00096, HG00097, HG00099, HG00100, HG00101, HG00102, HG00103, HG00105, HG00106, HG00107, HG00108, HG00109, HG00110, HG00111, HG00112, HG00113, HG00114, HG00115, HG00116, HG00117, HG00118, HG00119, HG00120, HG00121, HG00122, HG00123, HG00125, HG00126, HG00127, HG00128, HG00129, HG00130, HG00131, HG00132, HG00133, HG00136, HG00137, HG00138, HG00139, HG00140, HG00141, HG00142, HG00143, HG00145, HG00146, HG00148, HG00149, HG00150, HG00151, HG00154, HG00155, HG00157, HG00158, HG00159, HG00160, HG00171, HG00173, HG00174, HG00176, HG00177, HG00178, HG00179, HG00180, HG00181, HG00182, HG00183, HG00185, HG00186, HG00187, HG00188, HG00189, HG00190, HG00231, HG00232, HG00233, HG00234, HG00235, HG00236, HG00237, HG00238, HG00239, HG00240, HG00242, HG00243, HG00244, HG00245, HG00246, HG00250, HG00251, HG00252, HG00253, HG00254, HG00255, HG00256, HG00257, HG00258, HG00259, HG00260, HG00261, HG00262, ...]

[2504 rows x 0 

In [48]:
# Convert all the values to booleans
data = data.astype(bool)
data.dtypes

chr2:192381934_maternal     bool
chr2:192381934_paternal     bool
chr8:129199566_maternal     bool
chr8:129199566_paternal     bool
chr4:126752992_maternal     bool
                            ... 
chr5:158244083_paternal     bool
chr11:42844441_maternal     bool
chr11:42844441_paternal     bool
chr12:120832146_maternal    bool
chr12:120832146_paternal    bool
Length: 812, dtype: object

In [49]:
# Convert the DataFrame to a PyTorch tensor of long integers
data_tensor = torch.tensor(data.values, dtype=torch.bool)
torch.save(data_tensor, './data/data_tensor.pt')

In [38]:
# Split the data into train, validation, and test sets (assuming proportions remain the same)
train_size = int(0.8 * len(data_tensor))
val_size = int(0.1 * len(data_tensor))
test_size = len(data_tensor) - train_size - val_size
train_set, val_set, test_set = random_split(data_tensor, [train_size, val_size, test_size])

# Save the train, validation, and test sets
torch.save(train_set, './data/train_set.pt')
torch.save(val_set, './data/val_set.pt')
torch.save(test_set, './data/test_set.pt')